In [1]:
# Import all required packages.

from IPython.display import display
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# run the notebok to get data.

%run STRAIN_VariableGenerator.ipynb

In [ ]:
from DataProcessorScript import generate_variables

feature_list = [
    "activity_details",
    "sms_details",
    "audio_details",
    "conversation_details",
    "dark_details",
    "phonecharge_details",
    "phonelock_details",
    "gps_details"]

feature_count = len(feature_list)
feature_dict = generate_variables(feature_list=feature_list, restrict_seqlen=10)

In [ ]:
input_size_list = []
index_list = []
input_list = []

for key in feature_dict.keys():
    input_seq, target, indices = feature_dict[key]
    input_size_list.append(input_seq.shape[2])
    input_list.append(input_seq)
    index_list.append(indices)

print(input_size_list, len(input_list), len(index_list))

# Filling NaN Values.
for seq in input_list:
    nan_mask = seq != seq
    print(nan_mask.any())
    seq[nan_mask.detach()] = 0
    print(type(seq))
  

In [3]:
# Define ModelClass

class Strain(nn.Module):

    def __init__(self, input_size_list, num_classes=5, hidden_dim = 100):
        super(Strain, self).__init__()

        # since input_size_list contains a list of sizes of each feature, We initialize those many RNN cells, 1 for each feature.
        self.rnns = nn.ModuleList([nn.LSTM(input_size, hidden_dim, 1) for input_size in input_size_list])
        self.batch_norm = nn.BatchNorm2d(len(input_size_list)*100)
        self.linear = nn.Linear(len(input_size_list)*100, num_classes)

    def forward(self, input_list, index_list):
      
        if len(input_list) != len(index_list):
            print("mistamatching index and targets")
            return  
        
        output_list = []
        
        for idx, indices in enumerate(index_list):
            y_out, _ = self.rnns[idx](input_list[idx])
            y_out = torch.index_select(y_out, 0, index_list[idx])
            y_out = y_out.view(y_out.shape[0], -1)
            output_list.append(y_out)
          
        y_out = torch.cat(output_list, dim=1)
        y_out = self.batch_norm(y_out)
        y_out = self.linear(y_out)

        return y_out

In [4]:
input_seq, target, indices = generate_variables(feature_list=["activity_details"], restrict_seqlen=10)["activity_details"]

# declaring Network.
net = Strain(num_features=1, input_seq_len_list=[len(input_seq)])
criterion = nn.CrossEntropyLoss(size_average=True)
optimizer = optim.SGD(net.parameters(), 0.001)

In [ ]:
# Train the network

for i in range(2):
    optimizer.zero_grad()
    y_hat = net.forward(input_seq, indices=indices)
    loss = criterion(y_hat, target)
    loss.backward()
    optimizer.step()
    
    print(loss)

In [ ]:
print(indices)
print(input_seq.shape)

In [ ]:
x = torch.randn(300, 1, 100)
x = Variable(x)
y = torch.LongTensor([1,3,10])
y = Variable(y)
x = torch.index_select(x, 0, y)
print(x)

In [ ]:
print(y_hat)
print(target)